In [1]:
## Importing Required Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, accuracy_score

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import plot_tree

import optuna
from sklearn.ensemble import RandomForestClassifier
import time
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import log_loss
from sklearn.metrics import hinge_loss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import shap

In [2]:
path = "/Users/abhi/Downloads/randomairline.csv"
train_data = pd.read_csv(path).drop('Unnamed: 0', axis=1)

print(train_data.shape)
print(train_data.info())

(103904, 24)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 103904 non-null  int64  
 1   Gender                             103904 non-null  object 
 2   Customer Type                      103904 non-null  object 
 3   Age                                103904 non-null  int64  
 4   Type of Travel                     103904 non-null  object 
 5   Class                              103904 non-null  object 
 6   Flight Distance                    103904 non-null  int64  
 7   Inflight wifi service              103904 non-null  int64  
 8   Departure/Arrival time convenient  103904 non-null  int64  
 9   Ease of Online booking             103904 non-null  int64  
 10  Gate location                      103904 non-null  int64  
 11  Food and drink            

In [3]:
path = "/Users/abhi/Downloads/testingshiz.csv"
test_data = pd.read_csv(path).drop('Unnamed: 0', axis=1)

print(test_data.shape)
print(test_data.info())

(25976, 24)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25976 entries, 0 to 25975
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 25976 non-null  int64  
 1   Gender                             25976 non-null  object 
 2   Customer Type                      25976 non-null  object 
 3   Age                                25976 non-null  int64  
 4   Type of Travel                     25976 non-null  object 
 5   Class                              25976 non-null  object 
 6   Flight Distance                    25976 non-null  int64  
 7   Inflight wifi service              25976 non-null  int64  
 8   Departure/Arrival time convenient  25976 non-null  int64  
 9   Ease of Online booking             25976 non-null  int64  
 10  Gate location                      25976 non-null  int64  
 11  Food and drink                     25976 n

In [4]:
train_data.dropna(subset='Arrival Delay in Minutes', inplace= True)
print(train_data.isna().sum())

id                                   0
Gender                               0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Inflight wifi service                0
Departure/Arrival time convenient    0
Ease of Online booking               0
Gate location                        0
Food and drink                       0
Online boarding                      0
Seat comfort                         0
Inflight entertainment               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Inflight service                     0
Cleanliness                          0
Departure Delay in Minutes           0
Arrival Delay in Minutes             0
satisfaction                         0
dtype: int64


In [5]:
test_data.dropna(subset='Arrival Delay in Minutes', inplace= True)
print(test_data.isna().sum())

id                                   0
Gender                               0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Inflight wifi service                0
Departure/Arrival time convenient    0
Ease of Online booking               0
Gate location                        0
Food and drink                       0
Online boarding                      0
Seat comfort                         0
Inflight entertainment               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Inflight service                     0
Cleanliness                          0
Departure Delay in Minutes           0
Arrival Delay in Minutes             0
satisfaction                         0
dtype: int64


In [11]:
data = train_data.copy()

data['satisfaction'] = data['satisfaction'].apply(lambda x: 1 if x == 'satisfied' else 0)
print(data['satisfaction'].value_counts())


label_encoder = LabelEncoder()

data['Gender'] = label_encoder.fit_transform(data['Gender'])

data = pd.get_dummies(data, columns=['Customer Type', 'Type of Travel', 'Class'], drop_first=True)


data['Ease of Booking * Online Boarding'] = data['Ease of Online booking'] * data['Online boarding']

data['Flight Distance * Seat Comfort'] = data['Flight Distance'] * data['Seat comfort']


data['Delay per Distance'] = (
    (data['Departure Delay in Minutes'] + data['Arrival Delay in Minutes']) / 
    data['Flight Distance']).replace([float('inf'), -float('inf')], 0)

scaler = MinMaxScaler()

data['Delay per Distance'] = scaler.fit_transform(data[['Delay per Distance']])


service_features = [
    'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking',
    'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
    'Inflight entertainment', 'On-board service', 'Leg room service',
    'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']

# Compute the mean score for service quality
data['Service Quality Score'] = data[service_features].mean(axis=1)

# Normalize the new feature for consistent scaling
data['Service Quality Score'] = scaler.fit_transform(data[['Service Quality Score']])


data.drop(['Ease of Online booking','Online boarding','Flight Distance',
                 'Seat comfort','Departure Delay in Minutes','Arrival Delay in Minutes',
                'Flight Distance'], axis=1, inplace=True)


numeric_data = data.select_dtypes(include=['number'])


# Remove highly correlated features
corr_matrix = numeric_data.corr().abs()
upper = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]

data_reduced = data.drop(columns=to_drop)

0    58697
1    44897
Name: satisfaction, dtype: int64


In [12]:
data1 = test_data.copy()

data1['satisfaction'] = data1['satisfaction'].apply(lambda x: 1 if x == 'satisfied' else 0)
print(data1['satisfaction'].value_counts())


label_encoder = LabelEncoder()

data1['Gender'] = label_encoder.fit_transform(data1['Gender'])

data1 = pd.get_dummies(data1, columns=['Customer Type', 'Type of Travel', 'Class'], drop_first=True)


data1['Ease of Booking * Online Boarding'] = data1['Ease of Online booking'] * data1['Online boarding']

data1['Flight Distance * Seat Comfort'] = data1['Flight Distance'] * data1['Seat comfort']


data1['Delay per Distance'] = (
    (data1['Departure Delay in Minutes'] + data1['Arrival Delay in Minutes']) / 
    data1['Flight Distance']).replace([float('inf'), -float('inf')], 0)

scaler = MinMaxScaler()

data1['Delay per Distance'] = scaler.fit_transform(data1[['Delay per Distance']])


service_features = [
    'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking',
    'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
    'Inflight entertainment', 'On-board service', 'Leg room service',
    'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']

# Compute the mean score for service quality
data1['Service Quality Score'] = data1[service_features].mean(axis=1)

# Normalize the new feature for consistent scaling
data1['Service Quality Score'] = scaler.fit_transform(data1[['Service Quality Score']])


data1.drop(['Ease of Online booking','Online boarding','Flight Distance',
                 'Seat comfort','Departure Delay in Minutes','Arrival Delay in Minutes',
                'Flight Distance'], axis=1, inplace=True)


numeric_data1 = data1.select_dtypes(include=['number'])


# Remove highly correlated features
corr_matrix = numeric_data1.corr().abs()
upper = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]

data_reduced1 = data1.drop(columns=to_drop)

0    14528
1    11365
Name: satisfaction, dtype: int64


In [13]:
X = data_reduced.drop(columns=['satisfaction'])
y = data_reduced['satisfaction']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [14]:
# Define Optuna Objective Function

def objective(trial):
    param = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0)
    }
    
    model = LGBMClassifier(**param, random_state=42)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
    return score

# Optimize Hyperparameters with Optuna
start_time = time.time()
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Adjust n_trials based on available compute
end_time = time.time()

# Train LightGBM with Best Parameters
best_params = study.best_params
print("Best Parameters:", best_params)

final_model = LGBMClassifier(**best_params, random_state=42)
final_model.fit(X_train, y_train)

# Evaluate the Model
y_pred_light = final_model.predict(X_test)

accuracy_train = accuracy_score(y_test, y_pred_light)
conf_matrix_train = confusion_matrix(y_test, y_pred_light)
class_report_train = classification_report(y_test, y_pred_light)

# Print evaluation results
print("Accuracy:", accuracy_train)
print("Confusion Matrix:")
print(conf_matrix_train)
print("Classification Report:")
print(class_report_train)

[I 2024-11-24 16:35:23,829] A new study created in memory with name: no-name-04c6f6bf-3a31-47bb-b2e1-82a72249d6c4


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 28735, number of negative: 37565
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001473 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 988
[LightGBM] [Info] Number of data points in the train set: 66300, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.433409 -> initscore=-0.267957
[LightGBM] [Info] Start training from score -0.267957
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

[I 2024-11-24 16:35:25,898] Trial 0 finished with value: 0.9590588235294117 and parameters: {'num_leaves': 70, 'learning_rate': 0.16198248467442106, 'n_estimators': 242, 'max_depth': 4, 'min_child_samples': 25, 'colsample_bytree': 0.6731667626832069, 'subsample': 0.6927274234076806}. Best is trial 0 with value: 0.9590588235294117.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-11-24 16:35:36,403] Trial 1 finished with value: 0.9624012066365009 and parameters: {'num_leaves': 90, 'learning_rate': 0.0653370496093011, 'n_estimators': 496, 'max_depth': 10, 'min_child_samples': 47, 'colsample_bytree': 0.8270965794710655, 'subsample': 0.7834882886990084}. Best is trial 1 with value: 0.9624012066365009.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 28735, number of negative: 37565
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001358 seco

[I 2024-11-24 16:42:51,451] Trial 48 finished with value: 0.9617978883861238 and parameters: {'num_leaves': 87, 'learning_rate': 0.038429869746760964, 'n_estimators': 265, 'max_depth': 8, 'min_child_samples': 39, 'colsample_bytree': 0.7991170622605371, 'subsample': 0.7491288743551441}. Best is trial 36 with value: 0.963523378582202.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 28735, number of negative: 37565
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001442 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 988
[LightGBM] [Info] Number of data points in the train set: 66300, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.433409 -> initscore=-0.267957
[LightGBM] [Info] Start training from score -0.267957
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 28735, number of negative: 37565
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001319 seconds.
You can set `force_row_wise=true`

[I 2024-11-24 16:43:03,158] Trial 49 finished with value: 0.9613273001508296 and parameters: {'num_leaves': 97, 'learning_rate': 0.016778040779542536, 'n_estimators': 419, 'max_depth': 9, 'min_child_samples': 42, 'colsample_bytree': 0.6945233256913746, 'subsample': 0.7647982090559626}. Best is trial 36 with value: 0.963523378582202.


Best Parameters: {'num_leaves': 100, 'learning_rate': 0.02920679737421864, 'n_estimators': 432, 'max_depth': 10, 'min_child_samples': 33, 'colsample_bytree': 0.820926771629586, 'subsample': 0.7872916460054615}
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 35918, number of negative: 46957
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001630 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 988
[LightGBM] [Info] Number of data points in the train set: 82875, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.433400 -> initscore=-0.267994
[LightGBM] [Info] Start training from score -0.267994
Accuracy: 0.965007963704812
Confusion Matrix:
[[11504   236]
 [  489  8490]]
Classification Report:
              precision    recall  f1-score   support

In [16]:
X_test = data1.drop(["satisfaction"], axis = 1)

In [17]:
y_pred_test = final_model.predict(X_test)

accuracy_test = accuracy_score(data1["satisfaction"], y_pred_test)
conf_matrix_test = confusion_matrix(data1["satisfaction"], y_pred_test)
class_report_test = classification_report(data1["satisfaction"], y_pred_test)

# Print evaluation results
print("Accuracy:", accuracy_test)
print("Confusion Matrix:")
print(conf_matrix_test)
print("Classification Report:")
print(class_report_test)

Accuracy: 0.9632333062989997
Confusion Matrix:
[[14203   325]
 [  627 10738]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     14528
           1       0.97      0.94      0.96     11365

    accuracy                           0.96     25893
   macro avg       0.96      0.96      0.96     25893
weighted avg       0.96      0.96      0.96     25893

